# Surrounding Landscape

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [ ]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

biomass_raw = ee.Image(f"{data_folder}/ESA_CCI_2020")
age = ee.Image(f"{data_folder}/mapbiomas_2020").select("age")

method = 'mapbiomas'

## Surrounding mature forests

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1) at a 500m distance

In [ ]:
def get_mature_mask():
    
    if method == 'eu':
        
        transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
        return transition.eq(10).selfMask()
    
    elif method == 'mapbiomas':
        lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select([f"classification_{year}" for year in config.range_1985_2020])
        .byte()
        .rename([str(year) for year in config.range_1985_2020]))

        return lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()

def get_mature_biomass(biomass_raw, mature_mask):
    if method == 'eu':
        return biomass_raw.updateMask(mature_mask).rename("mature_biomass")
    elif method == 'mapbiomas':
        # keep only forests that are 1000m away from any non-forest area
        edge_detec = mature_mask.unmask(-1).zeroCrossing()
        distance_to_edge = edge_detec.fastDistanceTransform().sqrt() \
            .multiply(ee.Image.pixelArea().sqrt()).rename("distance_to_edge")
        distance_to_edge = distance_to_edge.gt(1000).selfMask().updateMask(mature_mask)
        return biomass_raw.updateMask(distance_to_edge).rename("mature_biomass")

def export_mature_biomass():
    mature_mask = get_mature_mask()
    sur_cover = mature_mask.unmask(0).focalMean(radius = 500, units = "meters").float().updateMask(age).rename("sur_cover")
    mature_biomass = get_mature_biomass(biomass_raw, mature_mask)

    export_image(mature_biomass, "mature_biomass_500", region = roi, folder = folder, scale = 500)
    export_image(sur_cover, "sur_cover", region = roi, folder = folder, scale = 30)

    return mature_biomass

export_mature_biomass()

NameError: name 'folder' is not defined

In [ ]:


ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
basemap = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas').filter(ee.Filter.date('2020-01-01', '2020-07-01')).first()
vis_planet = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}


map = geemap.Map()
map.addLayer(ecoregions, {}, "ecoregions", opacity = 0.01)
map.addLayer(basemap, vis_planet, 'basemap')
map.addLayer(mature_biomass_neighborhood_1km, {'min': 150, 'max': 250, 'palette': ['yellow', 'blue']}, "1km")
# map.addLayer(mature_biomass_neighborhood_5km, {'min': 150, 'max': 250, 'palette': ['yellow', 'blue']}, "5km")
# map.addLayer(mature_biomass_neighborhood_10km, {'min': 150, 'max': 250, 'palette': ['yellow', 'blue']}, "10km")
map
# export_image(mature_biomass_10km, "mature_biomass_10km", region = roi, scale = 10000)



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
TransitionMap = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes')

map = geemap.Map()
map.addLayer(TransitionMap.first(), {}, "TransitionMap")
map

In [ ]:
def get_mature_biomass_vector(mature_biomass):
    # there are time out issues with larger mature_biomass images.
    # to get around it, we reduce to vectors over the geometries of the biomes we are considering
    # and we exclude the pixels that are within the given search radius of the biome edges
    biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
    amaz_atla_pant = biome.filter(ee.Filter.inList('CD_Bioma', [1, 4, 6]))
    mature_biomass = mature_biomass.clip(amaz_atla_pant)

    mature_biomass_vector = mature_biomass.sample(region = mature_biomass.geometry(), scale = 1000, geometries = True)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = mature_biomass_vector,
        assetId = f"{data_folder}/{folder}/mature_biomass_vector_neighborhood",
        description = "mature_biomass_vector"
    )
        
    task.start()

# mature_biomass = ee.Image(f"{data_folder}/{folder}/mature_biomass")
mature_biomass = ee.Image("projects/amazon-forest-regrowth/assets/mature_biomass_neighborhood")

get_mature_biomass_vector(mature_biomass)


In [ ]:
def get_distance_to_nearest_mature(method):

    mature_biomass_vector = ee.FeatureCollection(f"{data_folder}/{folder}/mature_biomass_vector_neighborhood")
    # change radius depending on data resolution and patterns
    distance_to_nearest_mature = mature_biomass_vector.distance(searchRadius = 10000)\
        .toInt()

    export_image(distance_to_nearest_mature, "distance_to_nearest_mature_neighborhood", \
                 region = roi, folder = method, scale = 1000)

    return distance_to_nearest_mature

distance_to_nearest_mature = get_distance_to_nearest_mature(method)


In [ ]:
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000)))
    return buffered

def get_distances_vector_buffer(distance_to_nearest_mature, method):
    
    categorical = ee.Image(f"{data_folder}/categorical")
    biome_mask = categorical.select('biome').eq(1) \
    .Or(categorical.select('biome').eq(4)) \
    .Or(categorical.select('biome').eq(6))

    distance_to_nearest_mature = distance_to_nearest_mature \
        .addBands(biome_mask)

    distances_vector = distance_to_nearest_mature.stratifiedSample(
        numPoints = 50000, classBand = "biome", region = roi, geometries = True
    )

    distances_vector_buffer = distances_vector.map(buffer_distance)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = distances_vector_buffer,
        assetId = f"{data_folder}/{folder}/distances_vector_buffer",
        description = "distances_vector_buffer"
    )

    task.start()

distance_to_nearest_mature = ee.Image(f"{data_folder}/{folder}/distance_to_nearest_mature_neighborhood")
get_distances_vector_buffer(distance_to_nearest_mature, method)

In [29]:
def export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, method):

    nearest_mature_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                            crs = proj['crs'], crsTransform = proj['transform'])

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = nearest_mature_biomass,
        assetId = f"{data_folder}/{folder}/nearest_mature_biomass_neighborhood",
        description = "nearest_mature_biomass"
    )

    task.start()

# mature_biomass = ee.Image(f"{data_folder}/{folder}/mature_biomass_500")
mature_biomass = ee.Image("projects/amazon-forest-regrowth/assets/mature_biomass_neighborhood")
distances_vector_buffer = ee.FeatureCollection(f"{data_folder}/{folder}/distances_vector_buffer")
export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, method)

In [34]:
def export_nearest_mature_biomass_image(nearest_mature_biomass, method):

    # change geometries from polygons to points
    nearest_mature_biomass = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

    # Convert the FeatureCollection to an image
    nearest_mature_biomass_image = nearest_mature_biomass.reduceToImage(
        properties = ['first'],
        reducer = ee.Reducer.first()
    ).toInt16().rename('nearest_mature_biomass')

    export_image(nearest_mature_biomass_image, "nearest_mature_biomass_image_neighborhood", \
                 region = roi, folder = method, scale = 1000)

nearest_mature_biomass = ee.FeatureCollection(f"{data_folder}/{folder}/nearest_mature_biomass_neighborhood")
export_nearest_mature_biomass_image(nearest_mature_biomass, method)


In [ ]:
assetId = 'users/your-username/your-asset-name'
ee.data.deleteAsset(assetId)

## Amazon Quarters
Mean Biomass Value Per Climatic Region

In [ ]:
# Load the mature biomass image
mature_biomass = ee.Image(f"{data_folder}/mapbiomas/mature_biomass_500")

# Load the individual feature collections and combine them into a single FeatureCollection
regions = ee.FeatureCollection([
    ee.FeatureCollection(f"{data_folder}/raw/NE_outline_no_overlap").map(lambda f: f.set('sector', 1)),
    ee.FeatureCollection(f"{data_folder}/raw/NW_outline_no_overlap").map(lambda f: f.set('sector', 2)),
    ee.FeatureCollection(f"{data_folder}/raw/SW_outline_no_overlap").map(lambda f: f.set('sector', 3)),
    ee.FeatureCollection(f"{data_folder}/raw/SE_outline_no_overlap").map(lambda f: f.set('sector', 4)),
]).flatten()

# Function to get mean biomass per Feature
def create_biomass_image(feature):
    mean_biomass = mature_biomass.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=500
    )
    return feature.set(mean_biomass)

# Apply the function to each feature in the FeatureCollection
mean_biomass_features = regions.map(create_biomass_image)

# Helper function to create an image from a property in the feature collection
def create_property_image(feature_collection, property_name):
    return feature_collection.reduceToImage(
        properties=[property_name],
        reducer=ee.Reducer.first()
    ).rename(property_name)

# Create images for the 'mature_biomass' and 'sector' properties
mature_biomass_image = create_property_image(mean_biomass_features, 'mature_biomass')
sector_image = create_property_image(mean_biomass_features, 'sector')

# Combine the images into a single multi-band image
amazon_quarters = ee.Image.cat([mature_biomass_image, sector_image]).rename(["mean_mat_biomass_quarter", "quarter"])

# export_image(amazon_quarters, "amazon_quarters", region = roi, scale = 10000)
